In [48]:
!pip install -r ../../requirements.txt

[notice] A new release of pip is available: 23.0.1 -> 23.1.2


[notice] To update, run: python.exe -m pip install --upgrade pip

In [49]:
!pip install -r ../../requirements.dev.txt

[notice] A new release of pip is available: 23.0.1 -> 23.1.2

[notice] To update, run: python.exe -m pip install --upgrade pip

In [50]:
!pip install --no-cache-dir -e .

Obtaining file:///Z:/tmp/prism_v2_playground/work/BetaProgram

[notice] A new release of pip is available: 23.0.1 -> 23.1.2


[notice] To update, run: python.exe -m pip install --upgrade pip

  Installing build dependencies: started


  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'

## Issue Credential

Issuing a credential involves establishing a connection between the issuer and the holder, which is done by following the process outlined in `Example 01 - Connections`. Once the connection is established, the issuer will prepare and send a credential offer, which creates an issue record on both the issuer's and holder's agents. The holder will then retrieve the list of issue records, find the one they wish to accept, and notify the issuer of their acceptance. Finally, the issuer will issue the credential to the holder, completing the process.

This example presents the steps required to issue a credential using Atala PRISM Agents.

In [51]:
#🚨 Run this code cell to import requirements in the Kernel

import os
import time
import datetime
import base64
import json
import jwt
import re
import requests
from pprint import pprint
from dotenv import load_dotenv
from termcolor import colored,cprint

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.models import ErrorResponse
from prism_agent_open_api_specification_client.models import Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest, AcceptCredentialOfferRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, IssueCredentialRecordBaseClaims, IssueCredentialRecordPage, IssueCredentialRecordAllOfProtocolState
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.models import CreateManagedDidRequest, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.models import DIDOperationResponse, DIDResponse, DID, Service   
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did, publish_managed_did
from prism_agent_open_api_specification_client.api.did import get_did


  Building editable for prism-agent-open-api-specification-client (pyproject.toml): started
  Building editable for prism-agent-open-api-specification-client (pyproject.toml): finished with status 'done'
  Created wheel for prism-agent-open-api-specification-client: filename=prism_agent_open_api_specification_client-0.41.0-py3-none-any.whl size=2871 sha256=c9229265fe7403d59ae52574de36d249cfd87b8a2b85b3adfbd659cb5f1065fd
  Stored in directory: C:\Users\beckm\AppData\Local\Temp\pip-ephem-wheel-cache-oq81csx3\wheels\e4\2d\0c\b0fbb0963c476eae827cce75c1ea48aa09876d2853df295cc9
Successfully built prism-agent-open-api-specification-client
  Attempting uninstall: prism-agent-open-api-specification-client
    Found existing installation: prism-agent-open-api-specification-client 0.41.0
    Uninstalling prism-agent-open-api-specification-client-0.41.0:
      Successfully uninstalled prism-agent-open-api-specification-client-0.41.0


### Ultilitary functions

In [52]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def find_credential_record_by_state(client, state):
    credential_records: Response[IssueCredentialRecordPage] = get_credential_records.sync(client=client)

    for offer in credential_records.contents:
        if(offer.protocol_state == state):
            return offer
    return None 

def to_canonical_did(long_form):
    pattern = r"did:prism:[a-fA-F0-9]{64}"
    match = re.search(pattern, long_form)
    
    if match:
        return match.group()
    else:
        return None

def print_credential_record(credential_record):
    print(f"record_id:          {credential_record.record_id}")
    print(f"subject_id:         {credential_record.subject_id}")
    print(f"role:               {credential_record.role}")
    print(f"protocol_state:     {credential_record.protocol_state}")
    print(f"created_at:         {credential_record.created_at}")
    print(f"updated_at:         {credential_record.updated_at}")
    
def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")
    
troubleshooting_message = f'''
🚨 An issue occurred while attempting to interact with the PRISM Agent 🚨

- Check that the PRISM Agent you are trying to connect to is up and running, and that it is listening on the correct port. 
  You can try to connect to the Agent using a different tool to confirm that it is available. 
  (e.g. `curl --location '<host:port>/prism-agent/connections' --header 'apiKey: <key>'`) 
- Check if there are any network issues preventing the Notebook from connecting to the Agent. This can include firewalls, 
  proxies, and other network configurations.
- Ensure that the Agent URL is correct, and that the correct API Keys are provided in the variables.env file.
- If none of the above solutions work, check the logs of the Agent container to see if there are any more specific error 
  messages that can help diagnose the issue.'''

def preflight(url, api_key):
    try:
        endpoint = f'{url}/connections'
        headers = {'apiKey': api_key}
        response = requests.get(endpoint, headers=headers, timeout=15)
        if response.status_code == 200:
            print(f"URL ok: {url}")
        else:
            raise Exception(f"URL: {response.url} code: {response.status_code} content: {response.text}")
    except Exception as Ex:
        raise Exception(f'{troubleshooting_message}\n\nURL: {url}\nAPI Key: {api_key != ""}')

### Client instances

We will create two clients, one for the Issuer and one for the Holder, to establish a connection and perform the issue credential process.

⚠️ Remember to update the file variables.env with the URLs and API keys provided to you.


#### ⚠️ NOTE:
If your host operating system is a nix-based OS that is not OSX or Windows please ensure you load the `../BetaProgram/variables_linux.env` environment variables.  
To do this uncomment the following line in the cell below: `#load_dotenv("../BetaProgram/variables-linux.env")`.  
Otherwise you will encounter issues with errors such as `ConnectionRefusedError`, `ConnectError: [Errno <n>] Name or service not known`

In [53]:
load_dotenv("../BetaProgram/variables.env")
#load_dotenv("../BetaProgram/variables-linux.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

%xmode Minimal

preflight(issuerUrl, issuerApiKey)
preflight(holderUrl, holderApiKey)

%xmode Verbose

Exception reporting mode: Minimal
URL ok: http://host.docker.internal:8080/prism-agent
URL ok: http://host.docker.internal:8090/prism-agent
Exception reporting mode: Verbose


### Create connection

ℹ️ For details on this see "Example 01 - Connections"

In [54]:
print("Please wait...")

conn_request = CreateConnectionRequest()
conn_request.label = f'Issue credential {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
issuer_connection: Response[Connection] =  create_connection.sync(client=issuer_client,json_body=conn_request)

invitation = get_invitation_str(issuer_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)


issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

print(f"Issuer connection: {issuer_connection.connection_id}")
print(f"Holder connection: {holder_connection.connection_id}\n")

while (issuer_connection.state != 'ConnectionResponseSent' or 
       not(holder_connection.state == 'ConnectionResponseReceived' or holder_connection.state == 'ConnectionRequestSent')):
    issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Issuer State: {} / Holder State: {}".format(issuer_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")
print("\nIssuer connection:\n")
print_connection(issuer_connection)
print("\nHolder connection:\n")
print_connection(holder_connection)

Please wait...
Issuer connection: 4cca88ef-8605-4e29-a842-a09f71d9b4ac
Holder connection: 6ec0842c-e924-4417-926d-ab720f609ec5

Issuer State: InvitationGenerated / Holder State: ConnectionRequestPending
Issuer State: InvitationGenerated / Holder State: ConnectionRequestPending
Issuer State: ConnectionResponsePending / Holder State: ConnectionResponseReceived
Issuer State: ConnectionResponsePending / Holder State: ConnectionResponseReceived
Issuer State: ConnectionResponseSent / Holder State: ConnectionResponseReceived
Connection established between Issuer and Holder!

Issuer connection:

connection_id: 4cca88ef-8605-4e29-a842-a09f71d9b4ac
state:         ConnectionResponseSent
label:         Issue credential 2023-05-12 13:54:15
my_did:        did:peer:2.Ez6LSpvAM1zvz9dSnNxKMY1fM5VuNv5wZEbsQVwfnuVYuyRWe.Vz6MkvafFkdG4fVeeaLrUvsgM8uBFeDckQEkXZNuuRfyFW9FC.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwODAvZGlkY29tbSIsInIiOltdLCJhIjpbImRpZGNvbW0vdjIiXX0
their_did:     did:peer:

### Holder - Create an unpublished did:prism 

To issue a verifiable credential, the Holder must provide the credential subject to the Issuer. The credential subject is a DID identifier. We will use a long-form `did:prism` in this example. So the next step is to create an unpublished `did` on the Holder side. It is required to have a key with `purpose` equal to `assertionMethod` in the DID Document template.

ℹ️ For details on creating an unpublished `did` see "Example 02 - DID Registrar"

In [55]:
data = { 
    "documentTemplate": {
        "publicKeys": [
            {
                "id": "key1",
                "purpose": "authentication"
            },
            {
                "id": "key2",
                "purpose": "assertionMethod"
            }
        ],
        "services": [
            {
                "id": "did:prism:test1",
                "type": "LinkedDomains",
                "serviceEndpoint": [
                    "https://test1.com"
                ]
            }
        ]
    }
}

did_request = CreateManagedDidRequest.from_dict(data)
subject_did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=holder_client, json_body=did_request)

print(f"Subject did (credential subject): {subject_did.long_form_did}")

Subject did (credential subject): did:prism:901e82897cb99a2162326453f322c7d95b2f3ab315b8c7bf70198c6bdeaef520:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIFxomrtQC7ULMokasFYDK1KoYRQcg2b7DIf1IeY9gz-oGiD_81jHvoxCD9Z_naz1i-TiB259zV_ZbSieoXDw01rV9RJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiC78IJKcxJnLXXhSdGs8d4x1qQnc6pYBkqcKwYaw7UcKBogiP_H6EUjyQ33Z97zoBiSebK768LxEU0NljZuc6NE43USXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3wJOv_5qM038QtcRtAryDI197mcL5dseBwd9k7RjfBcaIJOplGGo87UZKQ0tru4MiwgrQUTvxutLdtxY5AYVQCOJGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v


### Issuer - Create and publish a did:prism 

To issue a verifiable credential, the Issuer must provide the issuing DID. The issuing DID is a DID identifier. We will use a canonical-form `did:prism` in this example. So the next step is to create and publish a `did` on the Issuer side. It is required to have a key with `purpose` equal to `assertionMethod` in the DID Document template.

ℹ️ For details on creating and publishing a `did` see "Example 02 - DID Registrar"

In [56]:
# Reusing the same did documentTemplate
issuing_did: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=issuer_client, json_body=did_request)

print(f"Issuing did: {issuing_did.long_form_did}")

operation_response : (DIDOperationResponse) = publish_managed_did.sync(client=issuer_client, did_ref=issuing_did.long_form_did)

print("Please wait...")

issuing_did : [DIDResponse] = get_did.sync(client=issuer_client, did_ref=operation_response.scheduled_operation.did_ref)

while (isinstance(issuing_did, ErrorResponse)):
    issuing_did : [DIDResponse] = get_did.sync(client=issuer_client, did_ref=operation_response.scheduled_operation.did_ref)
    print("Please wait...")
    time.sleep(10)
    
print(f"Issuing did published: {issuing_did.did.id}")

Issuing did: did:prism:d9007f2e87610668109cc449f605251c4f8c9121849e8751c0cddd01487eb24d:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIEBJ-8GpRKJuXZjI5CHrsI6d8E6WvFTLsiiwIH4QNKtlGiDGtD3SHvrQW3KPKlmDzDLOC0DWyo1PwphcrIrFTvBvqxJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiBM-Cm6lQOTx3LDf3OSWCumJRGAgSboCKF6a6MuenZlqBogbdq8pg3Ef0r9vmnivICi0bAaU0XaUv46V3syDpduR6cSXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgAscFpJngPJOf29IOsh7UNGpFqCvhNPUBpFTC4LvsC5saIG6CTL_9I6NUvSvNrktt39Nb1-lgL238KZvWUe3vWhQJGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v
Please wait...
Please wait...
Please wait...
Issuing did published: did:prism:d9007f2e87610668109cc449f605251c4f8c9121849e8751c0cddd01487eb24d


### Issuer - Create credential claim

Now that we have the DIDs ready, the next step is for the Issuer to create the credential claim object. The credential claim contains the attributes that will be part of the verifiable credential.

In [57]:
data = {
        "firstname": 'James',
        "lastname": 'Smith',
        "birthdate": '01/01/2000'
      }

credential_claims = IssueCredentialRecordBaseClaims().from_dict(data)

### Issuer - Send credential offer

Following, the Issuer creates a `CreateIssueCredentialRecordRequest`. It contains the `subject_id`, the `claims` and other metadata. This object is passed to the `create_credential_offer` endpoint.
The `create_credential_offer` call creates an `IssueCredentialRecord` on the issuer side, it also sends the credential offer to the Holder.

In [58]:
credential_offer = CreateIssueCredentialRecordRequest(subject_id=subject_did.long_form_did, 
                                                      issuing_did=issuing_did.did.id,
                                                      claims=credential_claims, 
                                                      connection_id=issuer_connection.connection_id,
                                                      validity_period=3600, 
                                                      automatic_issuance=False)

issuer_credential_record: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client,json_body=credential_offer)

print("\nIssuer credential record:\n")
print_credential_record(issuer_credential_record)


Issuer credential record:

record_id:          7c9edc58-0562-41e1-9c28-5a7a67ee3ee8
subject_id:         <prism_agent_open_api_specification_client.types.Unset object at 0x000002BC3C1E7BE0>
role:               Issuer
protocol_state:     OfferPending
created_at:         2023-05-12 17:54:42.175299+00:00
updated_at:         <prism_agent_open_api_specification_client.types.Unset object at 0x000002BC3C1E7BE0>


### Holder - Wait for credential offer

The Holder waits to receive the credential offer. When received, it will show up in the Holder's credential records list as a new entry with `protocol_state` equal to `OfferReceived`. The code below waits until a credential offer is received and takes the corresponding credential_record.

In [59]:
holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)

while(holder_credential_record == None):
    holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)
    time.sleep(1)
    
print("\nHolder credential record:\n")
print_credential_record(holder_credential_record)


Holder credential record:

record_id:          98e3c8b5-0ff9-4db4-a279-349e79d7df03
subject_id:         <prism_agent_open_api_specification_client.types.Unset object at 0x000002BC3C1E7BE0>
role:               Holder
protocol_state:     OfferReceived
created_at:         2023-05-12 17:54:44.087207+00:00
updated_at:         <prism_agent_open_api_specification_client.types.Unset object at 0x000002BC3C1E7BE0>


### Holder - Accept credential offer

Now the Holder uses the `accept_credential_offer` endpoint to accept the credential offer. It must provide the `record_id` of the offer. Accepting the credential offer tells the Issuer that the credential can be issued.

In [60]:
accept_credential = AcceptCredentialOfferRequest(subject_id=to_canonical_did(subject_did.long_form_did))

holder_credential_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client, record_id=holder_credential_record.record_id, json_body=accept_credential)
print("\nHolder credential record:\n")
print_credential_record(holder_credential_record)


Holder credential record:

record_id:          98e3c8b5-0ff9-4db4-a279-349e79d7df03
subject_id:         did:prism:901e82897cb99a2162326453f322c7d95b2f3ab315b8c7bf70198c6bdeaef520
role:               Holder
protocol_state:     RequestPending
created_at:         2023-05-12 17:54:44.087207+00:00
updated_at:         2023-05-12 17:54:45.582103+00:00


### Issuer - Wait for credential request

The Issuer waits to receive the credential request. When received, the credential record state will change to `RequestReceived` in the Issuer's credential records list. The code below waits until a credential request is received and takes the corresponding credential_record.

In [61]:
print("Please wait...")

issuer_credential_record = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_record.record_id)

print(f"Issuer credential record: {issuer_credential_record.record_id}")
print(f"Holder credential record: {holder_credential_record.record_id}\n")

while(issuer_credential_record.protocol_state != IssueCredentialRecordAllOfProtocolState.REQUESTRECEIVED):
    issuer_credential_record = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_record.record_id)
    holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)
    print(f"Issuer state: {issuer_credential_record.protocol_state} / Holder State: {holder_credential_record.protocol_state}")
    time.sleep(1)
    
print("\nIssuer credential record:\n")
print_credential_record(issuer_credential_record)

Please wait...
Issuer credential record: 7c9edc58-0562-41e1-9c28-5a7a67ee3ee8
Holder credential record: 98e3c8b5-0ff9-4db4-a279-349e79d7df03

Issuer state: OfferPending / Holder State: RequestPending
Issuer state: OfferSent / Holder State: RequestGenerated
Issuer state: OfferSent / Holder State: RequestGenerated
Issuer state: RequestReceived / Holder State: RequestGenerated

Issuer credential record:

record_id:          7c9edc58-0562-41e1-9c28-5a7a67ee3ee8
subject_id:         <prism_agent_open_api_specification_client.types.Unset object at 0x000002BC3C1E7BE0>
role:               Issuer
protocol_state:     RequestReceived
created_at:         2023-05-12 17:54:42.175299+00:00
updated_at:         2023-05-12 17:54:48.628565+00:00


### Issuer - Accept credential request (issue credential to holder)

The Issuer uses the `issue_credential` endpoint to issue the credential. It must provide the `record_id`. When issued the record state changes to `CredentialSent` and the credential is send to the Holder.

In [62]:
issuer_credential_record: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client, record_id=issuer_credential_record.record_id)
print("\nIssuer credential record:\n")
print_credential_record(issuer_credential_record)


Issuer credential record:

record_id:          7c9edc58-0562-41e1-9c28-5a7a67ee3ee8
subject_id:         <prism_agent_open_api_specification_client.types.Unset object at 0x000002BC3C1E7BE0>
role:               Issuer
protocol_state:     CredentialPending
created_at:         2023-05-12 17:54:42.175299+00:00
updated_at:         2023-05-12 17:54:50.300266+00:00


### Holder - Wait for credential

The Holder waits to receive the credential. When received, it will be added to the Holder's credential record and the `protocol_state` will be updated to `CredentialReceived`. The code below waits until a credential is received.

In [63]:
print("Please wait...")

holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)

print(f"Issuer credential record: {issuer_credential_record.record_id}")
print(f"Holder credential record: {holder_credential_record.record_id}\n")

while(holder_credential_record.protocol_state != IssueCredentialRecordAllOfProtocolState.CREDENTIALRECEIVED):
    issuer_credential_record = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_record.record_id)
    holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)
    print(f"Issuer state: {issuer_credential_record.protocol_state} / Holder State: {holder_credential_record.protocol_state}")
    time.sleep(1)

print("\nHolder credential record:\n")
print_credential_record(holder_credential_record)

Please wait...
Issuer credential record: 7c9edc58-0562-41e1-9c28-5a7a67ee3ee8
Holder credential record: 98e3c8b5-0ff9-4db4-a279-349e79d7df03

Issuer state: CredentialPending / Holder State: RequestGenerated
Issuer state: CredentialPending / Holder State: RequestSent
Issuer state: CredentialGenerated / Holder State: RequestSent
Issuer state: CredentialGenerated / Holder State: RequestSent
Issuer state: CredentialGenerated / Holder State: CredentialReceived

Holder credential record:

record_id:          98e3c8b5-0ff9-4db4-a279-349e79d7df03
subject_id:         did:prism:901e82897cb99a2162326453f322c7d95b2f3ab315b8c7bf70198c6bdeaef520
role:               Holder
protocol_state:     CredentialReceived
created_at:         2023-05-12 17:54:44.087207+00:00
updated_at:         2023-05-12 17:54:54.278569+00:00


### JWT Credential

The JWT Credential is available in the holder credential record. The website https://jwt.io/ can be used to decode the credential.

In [64]:
print(base64.b64decode(holder_credential_record.jwt_credential).decode())

eyJhbGciOiJFUzI1NksifQ.eyJpc3MiOiJkaWQ6cHJpc206ZDkwMDdmMmU4NzYxMDY2ODEwOWNjNDQ5ZjYwNTI1MWM0ZjhjOTEyMTg0OWU4NzUxYzBjZGRkMDE0ODdlYjI0ZDpDczBDQ3NvQ0Vsa0tCR3RsZVRFUUJFSlBDZ2x6WldOd01qVTJhekVTSUVCSi04R3BSS0p1WFpqSTVDSHJzSTZkOEU2V3ZGVExzaWl3SUg0UU5LdGxHaURHdEQzU0h2clFXM0tQS2xtRHpETE9DMERXeW8xUHdwaGNySXJGVHZCdnF4SlpDZ1JyWlhreUVBSkNUd29KYzJWamNESTFObXN4RWlCTS1DbTZsUU9UeDNMRGYzT1NXQ3VtSlJHQWdTYm9DS0Y2YTZNdWVuWmxxQm9nYmRxOHBnM0VmMHI5dm1uaXZJQ2kwYkFhVTBYYVV2NDZWM3N5RHBkdVI2Y1NYQW9IYldGemRHVnlNQkFCUWs4S0NYTmxZM0F5TlRack1SSWdBc2NGcEpuZ1BKT2YyOUlPc2g3VU5HcEZxQ3ZoTlBVQnBGVEM0THZzQzVzYUlHNkNUTF85STZOVXZTdk5ya3R0MzlOYjEtbGdMMjM4S1p2V1VlM3ZXaFFKR2pRS0QyUnBaRHB3Y21semJUcDBaWE4wTVJJTlRHbHVhMlZrUkc5dFlXbHVjeG9TYUhSMGNITTZMeTkwWlhOME1TNWpiMjB2Iiwic3ViIjoiZGlkOnByaXNtOjkwMWU4Mjg5N2NiOTlhMjE2MjMyNjQ1M2YzMjJjN2Q5NWIyZjNhYjMxNWI4YzdiZjcwMTk4YzZiZGVhZWY1MjA6Q3MwQ0Nzb0NFbGtLQkd0bGVURVFCRUpQQ2dselpXTndNalUyYXpFU0lGeG9tcnRRQzdVTE1va2FzRllESzFLb1lSUWNnMmI3RElmMUllWTlnei1vR2lEXzgxakh2b3hDRDlaX25hejFpLVRpQjI1OXpWX1piU

### ⚠️ Important Note
Keep the Holder credential record identifier at hand, it will be needed to run the next example:

In [65]:
print(f"\nHolder credential record_id: {holder_credential_record.record_id}\n")


Holder credential record_id: 98e3c8b5-0ff9-4db4-a279-349e79d7df03



### Decode verifiable credential

As an alternative to the website https://jwt.io/ below you will find the code to perform the verifiable credential decoding programmatically:

#### Unverified Decoding

In [66]:
try:
    jwt_decoded_id_token = jwt.decode(base64.b64decode(holder_credential_record.jwt_credential).decode(), options={"verify_signature": False})
    # print(jwt_decoded_id_token)
    print(json.dumps(jwt_decoded_id_token, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

{
  "iss": "did:prism:d9007f2e87610668109cc449f605251c4f8c9121849e8751c0cddd01487eb24d:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIEBJ-8GpRKJuXZjI5CHrsI6d8E6WvFTLsiiwIH4QNKtlGiDGtD3SHvrQW3KPKlmDzDLOC0DWyo1PwphcrIrFTvBvqxJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiBM-Cm6lQOTx3LDf3OSWCumJRGAgSboCKF6a6MuenZlqBogbdq8pg3Ef0r9vmnivICi0bAaU0XaUv46V3syDpduR6cSXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgAscFpJngPJOf29IOsh7UNGpFqCvhNPUBpFTC4LvsC5saIG6CTL_9I6NUvSvNrktt39Nb1-lgL238KZvWUe3vWhQJGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v",
  "sub": "did:prism:901e82897cb99a2162326453f322c7d95b2f3ab315b8c7bf70198c6bdeaef520:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIFxomrtQC7ULMokasFYDK1KoYRQcg2b7DIf1IeY9gz-oGiD_81jHvoxCD9Z_naz1i-TiB259zV_ZbSieoXDw01rV9RJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiC78IJKcxJnLXXhSdGs8d4x1qQnc6pYBkqcKwYaw7UcKBogiP_H6EUjyQ33Z97zoBiSebK768LxEU0NljZuc6NE43USXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIg3wJOv_5qM038QtcRtAryDI197mcL5dseBwd9k7RjfBcaIJOplGGo87UZKQ0tru4MiwgrQUTvxutLdtxY5AYVQCOJG

#### Verified Decoding

##### Resolve Issuer DID

In [67]:
did : [DIDResponse] = get_did.sync(client=holder_client, did_ref=jwt_decoded_id_token['iss'])

while (isinstance(did, ErrorResponse)):
    did : [DIDResponse] = get_did.sync(client=holder_client, did_ref=jwt_decoded_id_token['iss'])
    print("Please wait...")
    time.sleep(10)

print(did.to_dict())

{'did': {'id': 'did:prism:d9007f2e87610668109cc449f605251c4f8c9121849e8751c0cddd01487eb24d:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIEBJ-8GpRKJuXZjI5CHrsI6d8E6WvFTLsiiwIH4QNKtlGiDGtD3SHvrQW3KPKlmDzDLOC0DWyo1PwphcrIrFTvBvqxJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiBM-Cm6lQOTx3LDf3OSWCumJRGAgSboCKF6a6MuenZlqBogbdq8pg3Ef0r9vmnivICi0bAaU0XaUv46V3syDpduR6cSXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgAscFpJngPJOf29IOsh7UNGpFqCvhNPUBpFTC4LvsC5saIG6CTL_9I6NUvSvNrktt39Nb1-lgL238KZvWUe3vWhQJGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v', 'controller': 'did:prism:d9007f2e87610668109cc449f605251c4f8c9121849e8751c0cddd01487eb24d:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIEBJ-8GpRKJuXZjI5CHrsI6d8E6WvFTLsiiwIH4QNKtlGiDGtD3SHvrQW3KPKlmDzDLOC0DWyo1PwphcrIrFTvBvqxJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiBM-Cm6lQOTx3LDf3OSWCumJRGAgSboCKF6a6MuenZlqBogbdq8pg3Ef0r9vmnivICi0bAaU0XaUv46V3syDpduR6cSXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgAscFpJngPJOf29IOsh7UNGpFqCvhNPUBpFTC4LvsC5saIG6CTL_9I6NUvSvNrktt39Nb1-lgL238KZvW

##### Extract assertion public JWK

In [69]:
for verification_method in did.did.verification_method:
    if verification_method.id == did.did.assertion_method[0].uri:
        print('Issuer KeyId\n',verification_method.id)
        print('Issuer KeyId JWK\n',verification_method.public_key_jwk)
        issuer_jwk = verification_method.public_key_jwk

issuer_pubKey = jwt.algorithms.ECAlgorithm.from_jwk(json.dumps(issuer_jwk.to_dict()))

Issuer KeyId
 did:prism:d9007f2e87610668109cc449f605251c4f8c9121849e8751c0cddd01487eb24d:Cs0CCsoCElkKBGtleTEQBEJPCglzZWNwMjU2azESIEBJ-8GpRKJuXZjI5CHrsI6d8E6WvFTLsiiwIH4QNKtlGiDGtD3SHvrQW3KPKlmDzDLOC0DWyo1PwphcrIrFTvBvqxJZCgRrZXkyEAJCTwoJc2VjcDI1NmsxEiBM-Cm6lQOTx3LDf3OSWCumJRGAgSboCKF6a6MuenZlqBogbdq8pg3Ef0r9vmnivICi0bAaU0XaUv46V3syDpduR6cSXAoHbWFzdGVyMBABQk8KCXNlY3AyNTZrMRIgAscFpJngPJOf29IOsh7UNGpFqCvhNPUBpFTC4LvsC5saIG6CTL_9I6NUvSvNrktt39Nb1-lgL238KZvWUe3vWhQJGjQKD2RpZDpwcmlzbTp0ZXN0MRINTGlua2VkRG9tYWlucxoSaHR0cHM6Ly90ZXN0MS5jb20v#key2
Issuer KeyId JWK
 PublicKeyJwk(kty='EC', crv='secp256k1', x='TPgpupUDk8dyw39zklgrpiURgIEm6AihemujLnp2Zag', y='bdq8pg3Ef0r9vmnivICi0bAaU0XaUv46V3syDpduR6c', additional_properties={})


NotImplementedError: 

##### Verify Verifiable Credential against resolved Issuer DID and associated authentication public JWK

In [ ]:
try:
    jwt_verified_decoded_id_token = jwt.decode(base64.b64decode(holder_credential_record.jwt_credential).decode(), key = issuer_pubKey, algorithms=["ES256K"])
    # print(jwt_decoded_id_token)
    print(json.dumps(jwt_verified_decoded_id_token, indent=2))
    print("JWT Signature Verification Successful!")
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError) as e:
    print("[ERROR]", e)

NameError: name 'issuer_pubKey' is not defined

##### Verify verifiable credential (with modification) against resolved Issuer DID and associated authentication public JWK
> NOTE!! This should fail as we tampered with the verifiable credential

In [ ]:
jwt_tampered_id_token = base64.b64decode(holder_credential_record.jwt_credential).decode() + 'x'

try:
    jwt_tampered_decoded_id_token = jwt.decode(jwt_tampered_id_token, key = issuer_pubKey, algorithms=["ES256K"])
    # print(jwt_decoded_id_token)
    # print(json.dumps(jwt_tampered_decoded_id_token, indent=2))
except (jwt.ExpiredSignatureError, jwt.InvalidAudienceError, jwt.InvalidSignatureError) as e:
    print("[ERROR]", e)